In [0]:
import torch

In [0]:
dtype = torch.float
device = torch.device('cuda:0')

In [0]:
N, D_in, H, D_out = 64, 1000, 100, 10

In [0]:
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

In [0]:
w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)

In [0]:
learning_rate = 1e-6

for epoch in range(500):
    h = x.mm(w1)
    h_relu = h.clamp(min=0)
    y_preds = h_relu.mm(w2)

    loss = (y_preds - y).pow(2).sum().item()
    if epoch % 100 == 99:
        print(epoch, loss)

    grad_y_preds = 2.0 * (y_preds - y)
    grad_w2 = h_relu.t().mm(grad_y_preds)
    grad_h_relu = grad_y_preds.mm(w2.t())
    grad_h = grad_h_relu.clone()
    grad_h[h < 0] = 0
    grad_w1 = x.t().mm(grad_h)

    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

99 577.0562133789062
199 3.998274803161621
299 0.055831968784332275
399 0.0013692383654415607
499 0.0001432465942343697


In [0]:
N, D_in, H, D_out = 64, 1000, 100, 10

dtype = torch.float
device = torch.device('cuda:0')

x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6

for epoch in range(500):
    y_preds = x.mm(w1).clamp(min=0).mm(w2)

    loss = (y_preds - y).pow(2).sum()

    if epoch % 100 == 99:
        print(epoch, loss.item())

    loss.backward()

    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        w1.grad.zero_()
        w2.grad.zero_()

99 555.489990234375
199 3.146148204803467
299 0.02577080950140953
399 0.0004683990264311433
499 6.282467802520841e-05


In [0]:
N, D_in, H, D_out = 64, 1000, 100, 10

dtype = torch.float
device = torch.device('cuda:0')

x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H).cuda(),
    torch.nn.ReLU().cuda(),
    torch.nn.Linear(H, D_out).cuda()
)

loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(500):
    y_preds = model(x)
    loss = loss_fn(y_preds, y)
    if epoch % 100 == 99:
        print(epoch, loss.item())
    model.zero_grad()
    loss.backward()
    optimizer.step()

99 44.86338806152344
199 0.6487696170806885
299 0.0023034100886434317
399 1.7538208339828998e-05
499 1.7323526435575332e-07
